On perlmutter: use kernel pytorch-1.13.1

In [ ]:
!pip install nflows

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch

from numba import cuda 

from helpers.make_flow import *
from helpers.train_flow import *
from helpers.make_BC import *
from helpers.train_BC import *

seed = 8

In [ ]:
# computing
os.environ["CUDA_VISIBLE_DEVICES"]="0"

device = cuda.get_current_device()
device.reset()

# set the number of threads that pytorch will use
torch.set_num_threads(2)

# set gpu device
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu")
print( "Using device: " + str( device ), flush=True)

In [ ]:
# load in the data

SB_data = np.load(f"processed_data/od_SB.npy")
SR_data = np.load(f"processed_data/od_SR.npy")

print(f"SB data has shape {SB_data.shape}.")

In [ ]:
# train val test split
from sklearn.model_selection import train_test_split

SB_data_train, SB_data_val = train_test_split(SB_data, test_size=0.2, random_state=42)

print(f"SB train data has shape {SB_data_train.shape}.")
print(f"SB val data has shape {SB_data_val.shape}.")

## Flow creation and training

In [ ]:
# Create the flow

# This will be the upper subdirectory in saved_models/

num_layers = 1
num_hidden_features = 128
num_blocks = 15
flow_training_id = f"Masked_PRQ_AR_{num_layers}layers_{num_blocks}hidden_{num_blocks}blocks_{seed}seed"

flow_training_dir = os.path.join("models", f"{flow_training_id}/")
os.makedirs(flow_training_dir, exist_ok=True)

hyperparameters_dict = {"n_epochs":100,
                          "batch_size": 128,
                          "lr": 0.0001,
                          "weight_decay": 0.0001}


test_flow = make_masked_AR_flow(SB_data_train.shape[1]-1, num_layers, num_hidden_features, num_blocks)


In [ ]:

epochs, losses, losses_val, best_epoch = train_flow(test_flow, hyperparameters_dict, device, SB_data_train, SB_data_val, flow_training_dir, seed = seed)


In [ ]:
plt.figure()
plt.plot(epochs, losses, label = "loss")
plt.plot(epochs, losses_val, label = "val loss")
plt.legend()
plt.show()

print(best_epoch)

## BC training

In [ ]:
# sample from the flow

   
# Load in model
config_string = "epochs{0}_lr{1}_wd{2}_bs{3}".format(hyperparameters_dict["n_epochs"], hyperparameters_dict["lr"], hyperparameters_dict["weight_decay"], hyperparameters_dict["batch_size"])
checkpoint_path = os.path.join(flow_training_dir, f"{config_string}")
    
print(f"Loading in the best flow model model ...")
flow_best = torch.load(f"{checkpoint_path}_best_model.pt")
flow_best.to(device)

# freeze the trained model
for param in flow_best.parameters():
    param.requires_grad = False
flow_best.eval()

context_masses = torch.tensor(SR_data[:,-1].reshape(-1,1)).float().to(device)
SR_samples = flow_best.sample(1, context=context_masses).detach().cpu().numpy()
SR_samples = SR_samples.reshape(SR_samples.shape[0], 8)

SR_samples = np.hstack((SR_samples, np.reshape(SR_data[:,-1], (-1, 1))))
print(SR_samples.shape)
  

In [ ]:
for i in range(9):
    plt.figure()
    plt.hist(SR_data[:,i], bins = np.linspace(-5, 5, 30), density = True, label = "data", histtype = "step")
    plt.hist(SR_samples[:,i], bins = np.linspace(-5, 5, 30), density = True, label = "samples", histtype = "stepfilled", alpha = 0.5)

    #plt.xlabel(labels[i])
    plt.legend()
    plt.ylabel("Density")
    plt.show()

In [ ]:
hyperparameters_dict_BC = {"n_epochs":100,
                          "batch_size": 128,
                          "lr": 0.0001,
                         }


discriminate_datasets_kfold(flow_training_dir, SR_samples, SR_data, np.ones((SR_samples.shape[0],1)), np.ones((SR_data.shape[0],1)), 
                               SR_samples, SR_data, 9, hyperparameters_dict_BC, device, seed = seed)



